# Notable Telescope Apertures

This notebook will show how to use prysm to paint the apertures of notable telescopes.  Further modeling of these observatories will not be given here, and requries additional data (e.g., OPD maps or coefficients, masks) not widely available.  It is assumed the user sufficiently understands the components used to not require explanation of details.  All parameters are based on publically shown values and may be imprecise.  If you are a member of the science or engineering team for these systems, you should check all parameters against internal values.  

Links jump to telescopes:

- [HST](#HST)
- [JWST](#JWST)
- [LUVOIR-A](#LUVOIR-A)
- [LUVOIR-B](#LUVOIR-B)
- [HabEx-A](#HabEx-A)
- [HabEx-B](#HabEx-B)

In [ ]:
import numpy as np

from prysm.coordinates import make_xy_grid, cart_to_polar
from prysm.geometry import spider, circle, offset_circle
from prysm.segmented import composite_hexagonal_aperture

from matplotlib import pyplot as plt

## HST

HST has a primary mirror of diameter 2.4 m with 32% linear obscuration, and four spiders of 38 mm diameter rotated 45$^\circ$ from the cardinal axes.  There are an additional three small circular obscurations from pads used to secure the primary mirror.  The pads are 95% of the way to the edge of the mirror at ccw angles w.r.t. the x axis of -45, -165, and +75 degrees and have each a diameter of 150 mm.

In [ ]:
x, y = make_xy_grid(512, diameter=2.4)
r, t = cart_to_polar(x, y)

pm_od = circle(2.4/2, r)
pm_id = circle(2.4/2*.32, r)
mask = pm_od ^ pm_id # or pm_od & ~pm_id
plt.imshow(mask, cmap='gray')

After shading the primary, we now compute the spider and pad obscurations:

In [ ]:
spider_ = spider(4, 0.038, x, y, 45)
pads_r = 0.90*2.4/2
pad_angles = [np.radians(a) for a in [-45, -165, 75]]
pad_centers = [(pads_r*np.cos(a), pads_r*np.sin(a)) for a in pad_angles]
pads = [offset_circle(.075, x, y, c) for c in pad_centers]

# pads before this point is a list of the points INSIDE each circle.
# logical or, |, below produces a mask of "pixels inside ANY circle"
# these are an obscuration, so we invert it with ~
pads = (pads[0]|pads[1]|pads[2])
hst_pupil = mask & spider_ & ~pads
plt.imshow(hst_pupil, origin='lower', cmap='gray', extent=[x.min(), x.max(), y.min(), y.max()])
plt.title('Fully composited HST aperture')

In [ ]:
pad_centers

## JWST

<!-- This notebook will show how to use prysm to model the JWST.  The principle features of the JWST shown here are the shape of its pupil as well as both per-segment and aperture-wide wavefront errors.  Obviously JWST is a high complexity observatory, and each of its instruments are also complicated.  We will begin from the front of the observatory and not cover any radiometric or polychromatic topics in detail, users can compose these topics with what is presented here. -->

<!-- To avoid running afoul of any export restrictions, we will also use only publicly available, unlimited release values in constructing this model.  This means that if you actually want to JWST with a program inspired by what is here, you need to check and likely replace all values with the unpublicized versions. -->

JWST is a 2-ring segmented hexagonal design.  The central segment is missing, and there is a upside-down "Y" strut system to hold the secondary.  The segments are 1.32 m flat-to-flat, with 7 mm airgaps between.  We first paint the hexagons:

In [ ]:
x, y = make_xy_grid(512, diameter=6.6)

vtov, centers, windows, local_coords, local_masks, segment_ids, mask = composite_hexagonal_aperture(2, 1.32, 0.007, x, y, exclude=(0,))

plt.imshow(mask, origin='lower', cmap='gray', extent=[x.min(), x.max(), y.min(), y.max()])

And create the secondary struts, adding them to the mask:

In [ ]:
m1 = spider(1, .1, x, y, rotation=-120)
m2 = spider(1, .1, x, y, rotation=-60)
m3 = spider(1, .1, x, y, rotation=90)
spider_ = m1&m2&m3
plt.imshow(mask&spider_, origin='lower', cmap='gray', extent=[x.min(), x.max(), y.min(), y.max()])
plt.title('Fully composited JWST aperture')

## LUVOIR-A

LUVOIR-A (as of the 2018 new design) contains 120 hexagonal segments of flat-to-flat dimension 1.223 m.  Only the central segment is missing.  The strut design is essentially the same as JWST.  The first step in defining the aperture is to indicate which segment has which ID from prysm (which are deterministic) and mark the ones missing from the observatory for exclusion:

In [ ]:
x, y = make_xy_grid(512, diameter=15)

vtov, centers, windows, local_coords, local_masks, segment_ids, mask = composite_hexagonal_aperture(6, 1.223, 0.007, x, y, exclude=(0,))

fig, ax = plt.subplots(figsize=(10,10))
ax.imshow(mask, origin='lower', cmap='gray', extent=[x.min(), x.max(), y.min(), y.max()])
for center, id_ in zip(centers, segment_ids):
    plt.text(*center, id_)

Note that we have discarded all of the other information from the composition process, which will be identical to the previous invocation.  We now add the spider, pretty much the same as JWST:

In [ ]:
exclude = [
    0,
    91,
    109,
    97,
    103,
    115,
    121
]

*_, mask = composite_hexagonal_aperture(6, 1.223, 0.007, x, y, exclude=exclude)

m1 = spider(1, .2, x, y, rotation=-105)
m2 = spider(1, .2, x, y, rotation=-75)
m3 = spider(1, .2, x, y, rotation=90)
spider_ = m1&m2&m3
plt.imshow(mask&spider_, origin='lower', cmap='gray', extent=[x.min(), x.max(), y.min(), y.max()])
plt.title('Fully composited LUVOIR-A aperture')

## LUVOIR-B

LUVOIR-B is a smaller, unobscured co-design to LUVOIR-A using the same segment architecture.  We follow a similar two-step shading process to find which segment IDs must be excluded:

In [ ]:
x, y = make_xy_grid(512, diameter=8)

vtov, centers, windows, local_coords, local_masks, segment_ids, mask = composite_hexagonal_aperture(4, 0.955, 0.007, x, y, exclude=[])

fig, ax = plt.subplots(figsize=(10,10))
ax.imshow(mask, origin='lower', cmap='gray', extent=[x.min(), x.max(), y.min(), y.max()])
for center, id_ in zip(centers, segment_ids):
    plt.text(*center, id_)

In [ ]:
exclude = [
    37,
    41,
    45,
    49,
    53,
    57
]

*_, mask = composite_hexagonal_aperture(4, 0.955, 0.007, x, y, exclude=exclude)
plt.imshow(mask, origin='lower', cmap='gray', extent=[x.min(), x.max(), y.min(), y.max()])
plt.title('Fully composited LUVOIR-B aperture')

## HabEx-A

Habex architecture A is a 4m unobscured system, which is extremely simple to model:

In [ ]:
x, y = make_xy_grid(512, diameter=4)
r, t = cart_to_polar(x, y)
mask = circle(2, r)

plt.imshow(mask, origin='lower', cmap='gray', extent=[x.min(), x.max(), y.min(), y.max()])
plt.title('Fully composited HabEx A pupil')

## HabEx-B

Habex architecture B is an unobscured pupil of 6.5 m diameter based on a 3-ring fully populated hexagonal composition

In [ ]:
x, y = make_xy_grid(512, diameter=6.5)

vtov, centers, windows, local_coords, local_masks, segment_ids, mask = composite_hexagonal_aperture(3, 0.825, 0.007, x, y, exclude=[])

plt.imshow(mask, origin='lower', cmap='gray', extent=[x.min(), x.max(), y.min(), y.max()])
plt.title('Fully composited HabEx B pupil')